In [1]:
%load_ext dotenv
%dotenv

In [2]:
# https://huggingface.co/TechxGenus/Meta-Llama-3-70B-Instruct-GPTQ
LLAMA3_BASE_URL = 'https://corellm.wb.ru/llama3/v1'
LLAMA3_MODEL_NAME = 'Meta-Llama-3-70B-Instruct-GPTQ'


DEEPSEEK_BASE_URL = 'https://corellm.wb.ru/deepseek/v1'
DEEPSEEK_MODEL_NAME = 'DeepSeek-R1'


QWEN25_32B_BASE_URL = 'https://corellm.wb.ru/qwen25-coder-14b-instruct/v1/'
QWEN25_32B_MODEL_NAME = 'Qwen25-Coder-14B-Instruct'

In [3]:
import os

RWB_CORELLM_TOKEN = os.getenv("RWB_CORELLM_TOKEN")

In [4]:
SOURCE_ARTICLE_NAME = 'Exploring ChatGPT and its impact on society MA Haque, S Li'

In [5]:
import pandas as pd

df = pd.read_csv('pdf_extracted_text.csv')

In [30]:
import time
from openai import OpenAI

def process_pdf_with_llm(pdf_content, source_article_name, client, model_name, max_retries=3):
    if not pdf_content or pd.isna(pdf_content) or pdf_content.strip() == "":
        return "No content to process"
    
    for attempt in range(max_retries):
        try:
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "system",
                        "content": f"You are a helpful assistant. User will give you article. Find article context which references to '{source_article_name}'. Write only text from article that mentions or references this source. If no references found, write 'No references found'."
                    },
                    {
                        "role": "user",
                        "content": pdf_content[:8000],  # Limit content to avoid token limits
                    }
                ],
                model=model_name,
                stream=False,
                temperature=0.1
            )
            
            response = chat_completion.choices[0].message.content
            return response.strip()
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(2)  # Wait before retry
            else:
                return f"Error after {max_retries} attempts: {str(e)}"


In [32]:
import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed


base_url = DEEPSEEK_BASE_URL
model_name = DEEPSEEK_MODEL_NAME
digest_col_name = f"{model_name}_digest"

client = OpenAI(
    api_key=RWB_CORELLM_TOKEN,
    base_url=base_url,
)

def process_row(args):
    idx, pdf_content = args

    digest = process_pdf_with_llm(
        pdf_content=pdf_content,
        source_article_name=SOURCE_ARTICLE_NAME,
        client=client,
        model_name=model_name
    )

    return idx, digest

df[digest_col_name] = None

non_null_mask = df['pdf_content'].notna() & (df['pdf_content'].str.strip() != '')
rows_to_process = df[non_null_mask]

print(f"\nProcessing {len(rows_to_process)} rows with non-null PDF content...")
print("="*60)

args_list = [
    (idx, row['pdf_content'])
    for idx, row in rows_to_process.iterrows()
]

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(process_row, args): args for args in args_list}


    with tqdm.tqdm(total=len(futures), desc="Processing PDFs") as pbar:
        for future in as_completed(futures):
            idx, digest = future.result()
            df.at[idx, digest_col_name] = digest
            pbar.update(1)

print(f"\nProcessing complete")


Processing 30 rows with non-null PDF content...


Processing PDFs:   3%|▎         | 1/30 [00:29<14:17, 29.56s/it]


Row 8: Digest length: 20 characters


Processing PDFs:   7%|▋         | 2/30 [00:42<09:08, 19.59s/it]


Row 7: Digest length: 508 characters


...


Row 29: Digest length: 20 characters


Processing PDFs:  97%|█████████▋| 29/30 [03:29<00:06,  6.27s/it]


Row 30: Digest length: 19 characters


Processing PDFs: 100%|██████████| 30/30 [06:00<00:00, 12.01s/it]


Row 5: Digest length: 1561 characters

Processing complete!


In [33]:
import json

output_filename = 'pdf_extracted_text_with_digests.csv'
df.to_csv(output_filename, index=False)


In [35]:
print("DIGEST ANALYSIS for", model_name)
print("="*50)

print(f"Total digests generated: {df[digest_col_name].notna().sum()}")

references_found = df[df[digest_col_name].str.contains('No references found', na=False)].shape[0]
references_not_found = df[df[digest_col_name].notna() & ~df[digest_col_name].str.contains('No references found', na=False)].shape[0]

print(f"\n📈 Reference Analysis:")
print(f"  • References found: {references_not_found}")
print(f"  • No references found: {references_found}")
print(f"  • Success rate: {references_not_found/(references_not_found + references_found)*100:.1f}%")


📊 DIGEST ANALYSIS
Total digests generated: 30
Unique digest responses: 13

📈 Reference Analysis:
  • References found: 9
  • No references found: 21
  • Success rate: 30.0%

🔍 Examples of Found References:

Example 1:
  Row 0: No title...
  Digest: Based on the provided thesis text, **no references to "Exploring ChatGPT and its impact on society MA Haque, S Li"** are found. The excerpt includes the abstract, acknowledgments, table of contents, a...

Example 2:
  Row 1: No title...
  Digest: However, some previous studies also identify challenges, such as the risk of biased information and excessive reliance on technology (Basha, 2024; Haque & Li, 2024; Zhai et al., 2024)....

Example 3:
  Row 5: No title...
  Digest: The article references the source "Exploring ChatGPT and its impact on society MA Haque, S Li" in the following section:

**Section 1: Einleitung**  
"Allerdings gab es auch Fälle, in denen die Antwor...

❌ Examples of No References Found:

Example 1:
  Row 2: No title...

TypeError: Object of type int64 is not JSON serializable